In [1]:
import itertools
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
%run keras_sentinel.ipynb

2024-05-18 12:56:45.236829: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 12:56:46.105697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

test_year = 2019
test_seasons = [6]

train_years = [2017, 2018, 2020]
all_seasons = [3, 6, 9, 12]

In [3]:
for season in test_seasons:
    selected_data = []
    for year in train_years:
        file_key = f'processed_treesat_{year}{str(season).zfill(2)}.npy'
        filepath = data_dir.joinpath(file_key)
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)
        
    features = np.concatenate(selected_data, axis=0)
    
    matching_labels = np.vstack([labels]*len(train_years))
    
    model_name = f'conv_seasons_{"_".join(map(str, test_seasons))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    
    KerasModelCreator().run(
        matching_labels, features, model_path, epochs=10, overwrite=True)

Epoch 1/10
34007/34007 ━━━━━━━━━━━━━━━━━━━━ 113s 3ms/step - accuracy: 0.4158 - f1_majority: 0.2604 - loss: 0.0205 - mean_squared_error: 0.0410 - r2_score: 0.1087 - root_mean_squared_error: 0.2024 - val_accuracy: 0.5012 - val_f1_majority: 0.4297 - val_loss: 0.0179 - val_mean_squared_error: 0.0357 - val_r2_score: 0.1843 - val_root_mean_squared_error: 0.1890
Epoch 2/10
34007/34007 ━━━━━━━━━━━━━━━━━━━━ 113s 3ms/step - accuracy: 0.4996 - f1_majority: 0.4004 - loss: 0.0179 - mean_squared_error: 0.0358 - r2_score: 0.1817 - root_mean_squared_error: 0.1892 - val_accuracy: 0.5177 - val_f1_majority: 0.4623 - val_loss: 0.0175 - val_mean_squared_error: 0.0349 - val_r2_score: 0.1966 - val_root_mean_squared_error: 0.1868
Epoch 3/10
34007/34007 ━━━━━━━━━━━━━━━━━━━━ 114s 3ms/step - accuracy: 0.5163 - f1_majority: 0.4301 - loss: 0.0173 - mean_squared_error: 0.0346 - r2_score: 0.2005 - root_mean_squared_error: 0.1859 - val_accuracy: 0.5343 - val_f1_majority: 0.4791 - val_loss: 0.0168 - val_mean_squared_e

In [12]:
test_results = []
for season in test_seasons:
    file_key = f'processed_treesat_{test_year}{str(season).zfill(2)}.npy'
    filepath = data_dir.joinpath(file_key)
    with open(filepath, 'rb') as f:
        test_features = np.load(f)

    test_labels = KerasModelCreator().crop_y(labels)
    
    for i in range(test_features.shape[-1]):
        test_features[...,i] = KerasModelCreator().normalise_X(test_features[...,i])

    model_name = f'conv_seasons_{"_".join(map(str, test_seasons))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    model = tf.keras.models.load_model(model_path)
    result = model.evaluate(test_features, test_labels, verbose=0, return_dict=True)
    test_results.append(result)

In [13]:
test_results

[{'accuracy': 0.3891744911670685,
  'f1_majority': 0.3287714421749115,
  'loss': 0.022132743149995804,
  'mean_squared_error': 0.04426548629999161,
  'r2_score': 0.06594114750623703,
  'root_mean_squared_error': 0.21039365231990814}]

In [7]:
# import subprocess
# subprocess.run('sudo', 'shutdown', 'now')

In [8]:
# results_df = pd.DataFrame(results)
# results_df.insert(0, 'season_comb', season_combinations)
# results_df

In [11]:
# major_pred = y_pred.argmax(axis=1)
# major_true = y_true.argmax(axis=1)

# cr = classification_report(
#     major_true, major_pred, target_names=df[target].columns, 
#     output_dict=True, zero_division=np.nan
# )
# df_report = pd.DataFrame(cr).T
# df_report['support'] = df_report['support'].astype(int)

# display(HTML(df_report.round(decimals=2).to_html()))